4.1. Реализовать методы Эйлера, Рунге-Кутты и Адамса 4-го порядка в виде программ, задавая в качестве входных данных шаг сетки h. С использованием разработанного программного обеспечения решить задачу Коши для ОДУ 2-го порядка на указанном отрезке. Оценить погрешность численного решения с использованием метода Рунге – Ромберга и путем сравнения с точным решением. 

Вариант 23

Задача Коши:

$$ x^2 y'' + xy' - y -3x^2 = 0 $$ 


$$ y(1) = 3,   y'(1) = 2 $$

$$x \in [1,2], h = 0.1$$



Точное решение: 

$$ y = x^2 + x + \frac{1}{x}$$

In [1]:
def frange(start, stop, step):
    while start < stop:
        yield start
        start += step

In [2]:
a, b, h = 1, 2, 0.1

In [3]:
y, y1 = 3, 2

In [4]:
X = [x for x in frange(a, b+h, h)]

In [5]:
def f_accurate(x):
    return x**2 + x + 1/x

$$ y'' = 3 + y - \frac{y'}{x}

$$ y' = 3x + xy$$

In [6]:
def f11(x, y, y1):  #  y' == f(x, y)
    return 3 + y - y1/x

#### Явный метод Эйлера

In [8]:
def euler_vivid_y(x, y, f, h): # p == 1
    return y + h*f(x, y)

In [9]:
def error(y11, eps, h):  # eps E [X_k-1, X_k]
    return y11(eps) / 2 * h**2

#### Неявный метод Эйлера

In [10]:
def check_conditions_newton(f, f11, a, b, x_num):
    if f(a) * f(b) < 0 and f(x_num) * f11(x_num) > 0:
        return 1
    else:
        return 0

In [11]:
def Newton(X, eps, f, f11, a, b):
    if check_conditions_newton(f, f11, a, b, X):
        error = 10 * eps
        itera = 0
        while (error > eps):
            X_new = X - (f(X)/f1(X))
            error = abs(X_new - X)
            X = X_new
            itera += 1
        return X_new, itera
    else:
        print("Contitions are broken")

In [12]:
def euler_vauge_y(x1, y, f, h): # p == 1
    y1 = y + h * f(x1, y1)

#### Метод Эйлера-Коши

In [13]:
def cauche_euler(x, y, f, h):  # p = 2
    y_tmp = y + h * f(x, y)
    return y + h * (f(x, y) + f(x+h, y_tmp)) / 2

#### Неявный метод Эйлера-Коши

In [14]:
def vauge_cauche_euler(x, y, f, h):
    y1 = y + h* (f(x, y) + f(x+h, y1)) / 2

#### Метод Эйлера-Коши с итерационной обработкой

In [15]:
def cauche_euler_iter(x, y, f, h):  # p == 2
    y_tmp = y + h * f(x, y)
    for i in range(4):
        y_res = y + h * (f(x, y) + f(x+h, y_tmp)) / 2
        y_tmp = y_res
    return y_res

#### Первый улучшенный метод Эйлера

In [16]:
def cauche_euler_improved(x, y, f, h): # p == 2
    y_half = y + h/2 * f(x, y)
    return y + h * f(x + h / 2, y_half)

#### Методы Рунге-Кутты

#### Метод Рунге-Кутты третьего порядка точности

In [17]:
def runge_knutta_3(x, y, f, h):
    K1 = h * f(x, y)
    K2 = h * f(x + 1/2 * h, y + 1/3 * K1)
    K3 = h * f(x + 3/2 * h, y + 2/3 * K2)
    delta_y =  (K1 + 3 * K3) / 4
    return y + delta_y

#### Метод Рунге-Кутты четвертого порядка точности

In [18]:
def runge_knutta_4(x, y, f, h):
    K1 = h * f(x, y)
    K2 = h * f(x + h / 2, y + K1 / 2)
    K3 = h * f(x + h / 2, y +  K2 / 2)
    K4 = h * f(x + h, y + K3)
    delta_y =  (K1 + 2* K2 + 2 * K3 + K4) / 6
    return y + delta_y

### Универсальная функция

In [19]:
def func(a, b, z ,step, f11, fun):
    data = []
    for x in frange(a, b + step, step):
        data.append(x, fun(x, z, f11, h))